In [1]:
cd("/home/gridsan/aramadhan/Oceananigans.jl/")
pwd()

"/home/gridsan/aramadhan/Oceananigans.jl"

In [2]:
using Pkg
Pkg.activate(".")

"/home/gridsan/aramadhan/Oceananigans.jl/Project.toml"

In [3]:
using Oceananigans, Oceananigans.Operators

In [4]:
using Adapt
Adapt.adapt_structure(to, A::SubArray{<:Any,<:Any,AT}) where {AT} = SubArray(adapt(to, parent(A)), adapt.(Ref(to), parentindices(A)))

In [5]:
using Random, Statistics
using FFTW
using CuArrays

In [6]:
Nx, Ny, Nz = 8, 8, 8
Lx, Ly, Lz = 1, 1, 1
R = rand(MersenneTwister(0), Nx, Ny, Nz)
R = R .- mean(R)
R = CuArray(R)

8×8×8 CuArray{Float64,3}:
[:, :, 1] =
  0.319189  -0.14263     0.0714293  …   0.447233  -0.320343    0.043884 
  0.405898   0.468758    0.363821       0.29666   -0.192313   -0.0275754
 -0.339892   0.0813535   0.463341      -0.380135  -0.308051    0.0979165
 -0.327129   0.0348312   0.263232      -0.390189   0.369123    0.286888 
 -0.225578  -0.244422   -0.0351544     -0.424904   0.150463   -0.455336 
 -0.300982   0.405588   -0.442091   …   0.272216   0.0822544  -0.0216966
 -0.462156  -0.337422   -0.151329      -0.399635   0.128237    0.160793 
 -0.436189   0.15099     0.263144       0.333617   0.230546    0.337798 

[:, :, 2] =
 -0.335351    -0.112398   -0.409666  …  -0.267658   -0.276227   -0.0693668 
  0.434399    -0.0251016   0.444426     -0.430831   -0.246833    0.482016  
  0.279268    -0.436235    0.27073      -0.39124     0.295758    0.492683  
  0.00134654  -0.02451    -0.327049      0.380707    0.211811    0.00313643
  0.117698    -0.0360611  -0.424435      0.0102109   0.024118

In [7]:
R2 = copy(R)
R2 .= cat(R2[:, 1:2:Ny, :], R2[:, Ny:-2:2, :]; dims=2)
R2 .= cat(R2[:, :, 1:2:Nz], R2[:, :, Nz:-2:2]; dims=3);

In [8]:
@inline W(M, k) = exp(-2im*π*k/M)

ω_4Ny_ky⁺ = reshape(W.(4Ny, 0:Ny-1), 1, Ny, 1) |> CuArray
ω_4Ny_ky⁻ = reshape(W.(4Ny, 0:-1:-(Ny-1)), 1, Ny, 1) |> CuArray
ω_4Nz_kz⁺ = reshape(W.(4Nz, 0:Nz-1), 1, 1, Nz) |> CuArray
ω_4Nz_kz⁻ = reshape(W.(4Nz, 0:-1:-(Nz-1)), 1, 1, Nz) |> CuArray

R3 = fft(R2, [2, 3])

r_inds = [1, collect(Ny:-1:2)...] |> CuArray
R3⁻ = view(R3, 1:Nx, r_inds, 1:Nz)

R4 = zeros(Nx, Ny, Nz) |> CuArray
@. R4 = 2 * real(ω_4Nz_kz⁺ * (ω_4Ny_ky⁺ * R3 + ω_4Ny_ky⁻ * R3⁻));

In [9]:
Array(R4) ≈ FFTW.r2r(Array(R), FFTW.REDFT10, [2, 3])

true

In [10]:
R4 = fft(R4, 1)

8×8×8 CuArray{Complex{Float64},3}:
[:, :, 1] =
 -2.75335e-14+0.0im        16.3717+0.0im      …   8.12933+0.0im    
      19.5186+4.81314im    12.2361-13.9949im     0.595239-22.4995im
       3.1991+13.9879im  -0.498285-11.4825im     -13.0339+14.8646im
      -14.617-15.9059im   -5.18881-12.2657im     -3.82183-6.10438im
      -7.1959+0.0im       -16.3921+0.0im           17.487+0.0im    
      -14.617+15.9059im   -5.18881+12.2657im  …  -3.82183+6.10438im
       3.1991-13.9879im  -0.498285+11.4825im     -13.0339-14.8646im
      19.5186-4.81314im    12.2361+13.9949im     0.595239+22.4995im

[:, :, 2] =
 -16.0768+0.0im      -4.38193+0.0im      …   1.95911+0.0im    
   6.2655-4.10268im   8.08108-8.56454im     -12.0896-16.5191im
 -22.4994+9.64744im   6.04621-8.56363im      6.15343+13.6236im
 -10.5211-2.95195im  -10.9848-2.51886im     -4.35911+8.02747im
 -24.9668+0.0im      -2.12844+0.0im         -5.84153+0.0im    
 -10.5211+2.95195im  -10.9848+2.51886im  …  -4.35911-8.02747im
 -22.4994-9.64744i

In [11]:
is = reshape(1:Nx, Nx, 1, 1)
js = reshape(1:Nx, 1, Ny, 1)
ks = reshape(1:Nx, 1, 1, Nz)

kx² = (@. (2sin((is-1)*π/Nx)    / (Lx/Nx))^2) |> CuArray
ky² = (@. (2sin((js-1)*π/(2Ny)) / (Ly/Ny))^2) |> CuArray
kz² = (@. (2sin((ks-1)*π/(2Nz)) / (Lz/Nz))^2) |> CuArray

@. R4 = -R4 / (kx² + ky² + kz²)

R4[1, 1, 1] = 0

R4

8×8×8 CuArray{Complex{Float64},3}:
[:, :, 1] =
       0.0+0.0im          -1.68029-0.0im        …   -0.0330116-0.0im      
 -0.520631-0.128384im    -0.259054+0.296291im      -0.00209778+0.0792943im
 -0.024993-0.109281im   0.00361749+0.0833612im       0.0348262-0.0397176im
 0.0668941+0.0727925im   0.0227327+0.0537372im      0.00822313+0.0131343im
  0.028109-0.0im         0.0616841-0.0im            -0.0348168-0.0im      
 0.0668941-0.0727925im   0.0227327-0.0537372im  …   0.00822313-0.0131343im
 -0.024993+0.109281im   0.00361749-0.0833612im       0.0348262+0.0397176im
 -0.520631+0.128384im    -0.259054-0.296291im      -0.00209778-0.0792943im

[:, :, 2] =
   1.65001-0.0im          0.224866-0.0im        …  -0.00765279-0.0im      
 -0.132649+0.0868591im    -0.14183+0.150315im        0.0411924+0.0562849im
  0.163343-0.0700392im  -0.0409949+0.0580637im      -0.0160246-0.035478im 
 0.0460938+0.0129328im   0.0461551+0.0105836im      0.00918656-0.0169174im
 0.0939509-0.0im        0.00772609-0.0im

In [12]:
R4 = ifft(R4, 1)

8×8×8 CuArray{Complex{Float64},3}:
[:, :, 1] =
  -0.116169-1.73472e-17im  …    0.00175933+8.67362e-19im
 -0.0702269-4.14001e-17im      -0.00800866+1.89616e-18im
  0.0600559-3.25487e-17im       -0.0337251+1.61696e-19im
  0.0828541-1.95672e-17im       -0.0242185-2.43023e-18im
     0.1107+3.46945e-18im      -0.00130334-2.60209e-18im
    0.11784+3.6196e-17im   …     0.0283188-1.61433e-19im
 -0.0405321+4.64265e-17im     -0.000645105+1.57303e-18im
  -0.144522+2.47714e-17im       0.00481096+6.95504e-19im

[:, :, 2] =
 0.237193+3.46945e-18im  …  0.00905816+8.67362e-19im
 0.162779+1.05798e-17im     0.00518537+1.0244e-18im 
 0.158678-7.7274e-18im      -0.0138249+9.53943e-19im
 0.190955-2.05131e-17im     -0.0238694-2.09548e-19im
  0.28047-1.04083e-17im     -0.0161313-2.60209e-18im
 0.261256+1.02369e-17im  …  0.00778813-2.75912e-18im
 0.195641+1.46663e-17im      0.0227763+7.8078e-19im 
 0.163041-3.03569e-19im     0.00136489+1.94427e-18im

[:, :, 3] =
 -0.0932523-4.33681e-19im  …  -0.00612046+1.355

In [13]:
r_y_inds = [1, collect(Ny:-1:2)...] |> CuArray
r_z_inds = [1, collect(Nz:-1:2)...] |> CuArray

R4⁻⁺ = view(R4, 1:Nx, r_y_inds, 1:Nz)
R4⁺⁻ = view(R4, 1:Nx, 1:Ny, r_z_inds)
R4⁻⁻ = view(R4, 1:Nx, r_y_inds, r_z_inds)

mask_ky = ones(1, Ny, 1)  |> CuArray
mask_kz = ones(1, 1, Nz)  |> CuArray

mask_ky[1] = 0
mask_kz[1] = 0

R5 = zeros(Complex{Float64}, Nx, Ny, Nz) |> CuArray
@. R5 = 1/4 *  ω_4Ny_ky⁻ * ω_4Nz_kz⁻ * ((R4 - mask_ky * mask_kz * R4⁻⁻) - im*(mask_kz * R4⁺⁻ + mask_ky * R4⁻⁺))

8×8×8 CuArray{Complex{Float64},3}:
[:, :, 1] =
 -0.0290422-4.33681e-18im  …  -0.0637881+0.0131367im 
 -0.0175567-1.035e-17im       -0.0862771+0.0151202im 
   0.015014-8.13718e-18im     -0.0663443+0.00460025im
  0.0207135-4.89181e-18im     -0.0524197+0.00425369im
  0.0276749+8.67362e-19im     -0.0349649+0.00662274im
    0.02946+9.04899e-18im  …  -0.0297334+0.0131327im 
  -0.010133+1.16066e-17im     -0.0349943+0.00679636im
 -0.0361304+6.19285e-18im      -0.045088+0.0101949im 

[:, :, 2] =
 0.0596959+0.00384075im   …  -0.00133789-0.00200612im
 0.0404635+0.00517111im       -0.0117597-2.05682e-5im
 0.0392038+0.00624844im      -0.00317411-0.00100566im
 0.0477609+0.00459025im       0.00743793-0.00389522im
 0.0686976+0.0140443im        0.00510589-0.00142718im
  0.064407+0.0109929im    …    0.0188606+0.00305531im
 0.0493198+0.00275867im        0.0199752+0.00395622im
 0.0414435+0.000580159im       0.0112155-0.00410216im

[:, :, 3] =
  -0.0231579-0.00501204im  …  0.0165894-0.00165381im 
  -0.0230

In [14]:
R6 = ifft(R5, [2, 3])

8×8×8 CuArray{Complex{Float64},3}:
[:, :, 1] =
 -0.00159679+8.50688e-19im  …  -0.000855049+7.47277e-19im
  -0.0033367-6.68653e-20im      -0.00331986+1.55612e-19im
 0.000688929-9.52146e-19im     -0.000804495-6.75224e-19im
  0.00297018-8.22288e-19im       0.00116887-8.07865e-19im
  0.00390202-5.13787e-19im       0.00290069-2.64389e-19im
  0.00429916-8.85952e-20im  …    0.00162123+2.08196e-19im
  0.00431429-2.98847e-20im       0.00244499+4.08936e-19im
  0.00250602+3.29945e-19im      -0.00016344+3.16922e-19im

[:, :, 2] =
 0.000485045+3.72173e-19im  …   -0.00201165+1.09411e-19im
 -0.00083052+2.25664e-20im      -0.00284027-9.34894e-20im
 -0.00185301-3.49817e-19im     -0.000305006-3.50956e-19im
 0.000675598-5.2621e-19im      -0.000800465-2.36223e-19im
  0.00213595-3.47599e-19im     -0.000185119-2.48888e-19im
  0.00155191+3.11483e-19im  …    0.00176866-4.91381e-21im
 0.000621566+4.64445e-19im      0.000912773+2.94444e-19im
 0.000729863+4.33295e-19im      -0.00174163+4.41902e-19im

[:, :, 3] =

In [15]:
R7 = zeros(Nx, Ny, Nz) |> CuArray

p_y_inds = [1:2:Ny..., Ny:-2:2...] |> CuArray
p_z_inds = [1:2:Nz..., Nz:-2:2...] |> CuArray
R7_p = view(R7, 1:Nx, p_y_inds, p_z_inds)

@. R7_p = real(R6)

8×8×8 view(::CuArray{Float64,3}, 1:8, [1, 3, 5, 7, 8, 6, 4, 2], [1, 3, 5, 7, 8, 6, 4, 2]) with eltype Float64:
[:, :, 1] =
 -0.00159679   -0.000847561   0.0010956    …   0.00150972   -0.000855049
 -0.0033367    -0.00244706    0.00145344       0.0015242    -0.00331986 
  0.000688929  -0.00166385    0.00356681       0.00288025   -0.000804495
  0.00297018    0.000428707   0.00151528       0.00293817    0.00116887 
  0.00390202    0.00229001    0.00121111       0.00193082    0.00290069 
  0.00429916    0.00315583    0.00308235   …   0.00297069    0.00162123 
  0.00431429    0.000937283   0.000915291     -0.000608081   0.00244499 
  0.00250602   -0.0015412    -0.00112641      -0.0016028    -0.00016344 

[:, :, 2] =
  0.000485045   0.000284204  0.000769018  …   0.00041781   -0.00201165 
 -0.00083052   -0.00184308   0.0022492        0.00107628   -0.00284027 
 -0.00185301   -0.00178016   0.00147633       0.000310334  -0.000305006
  0.000675598  -0.00114127   0.000394467      0.000132043  -0.00

In [17]:
function ∇²!(grid::RegularCartesianGrid, f, ∇²f)
    for k in 1:grid.Nz, j in 1:grid.Ny, i in 1:grid.Nx
        @inbounds ∇²f[i, j, k] = ∇²(grid, f, i, j, k)
    end
end

grid = RegularCartesianGrid((Nx, Ny, Nz), (Lx, Ly, Lz))
fbcs = ChannelBCs()

ϕ   = CellField(Float64, GPU(), grid)
∇²ϕ = CellField(Float64, GPU(), grid)

data(ϕ) .= real.(R7)

fill_halo_regions!(grid, (:T, fbcs, ϕ.data))
∇²!(grid, ϕ, ∇²ϕ)

fill_halo_regions!(grid, (:T, fbcs, ∇²ϕ.data))

data(∇²ϕ) ≈ R

true

In [59]:
A = rand(Complex{Float64}, 4)

4-element Array{Complex{Float64},1}:
 0.6018868078076858 + 0.3778857653180918im 
 0.9985572167715593 + 0.2075139900281715im 
 0.8029148594734494 + 0.13310879132086284im
 0.3355425244394772 + 0.902609142796889im  

In [60]:
B = rand(4)

4-element Array{Float64,1}:
 0.0350410684345106 
 0.2904072189634592 
 0.20722290392817877
 0.4371161404538675 

In [61]:
FFT! = plan_fft!(A)

FFTW in-place forward plan for 4-element array of Complex{Float64}
(dft-direct-4 "n2fv_4_avx2_128")

In [62]:
FFT! * B

4-element Array{Complex{Float64},1}:
   0.9697873317800161 + 0.0im                
 -0.17218183549366817 + 0.14670892149040826im
 -0.48525938705463734 + 0.0im                
 -0.17218183549366817 - 0.14670892149040826im

In [63]:
B

4-element Array{Float64,1}:
 0.0350410684345106 
 0.2904072189634592 
 0.20722290392817877
 0.4371161404538675 

In [46]:
A = rand(Complex{Float64}, 4) |> CuArray

4-element CuArray{Complex{Float64},1}:
  0.8880690027275071 + 0.8423715808367269im 
  0.2356147612959214 + 0.6733324497099495im 
 0.12433199107670045 + 0.925790007573414im  
  0.2249226840800238 + 0.39188113946557457im

In [47]:
B = rand(4) |> CuArray

4-element CuArray{Float64,1}:
 0.05464150338792462
 0.6706693686629823 
 0.12889635541186295
 0.2563894700350746 

In [48]:
FFT! = plan_fft!(A)

CUFFT in-place d.p. complex forward plan for 4-element CuArray of Complex{Float64}

In [49]:
FFT! * A

4-element CuArray{Complex{Float64},1}:
 1.4729384391801528 + 2.833375177585665im  
 1.0451883218951816 - 0.09411050395258468im
 0.5518635484282624 + 0.7029479992346168im 
 0.4822857014064317 - 0.0727263495207895im 

In [50]:
B

4-element CuArray{Float64,1}:
 0.05464150338792462
 0.6706693686629823 
 0.12889635541186295
 0.2563894700350746 